### Stack Class 구현

In [44]:
class Stack:
    def __init__(self):
        self.top = []
        
    def __str__(self):
        return str(self.top) # 역순으로 출력하고자 하면 return str(self.top[::-1])
        
    def isEmpty(self) : return len(self.top) == 0
    def size(self) : return len(self.top)
    def clear(self) : self.top = []
        
    def push(self, item):
        self.top.append(item)
        
    def pop(self):
        if not self.isEmpty():
            return self.top.pop(-1)
    
    def peek(self):
        if not self.isEmpty():
            return self.top[-1]

In [12]:
odd = Stack()
even = Stack()

for i in range(10):
    if i%2 == 0 : even.push(i)
    else : odd.push(i)

print(even)
print(odd)
print(even.peek())
print(odd.peek())
for _ in range(2) : even.pop()
print(even)

[0, 2, 4, 6, 8]
[1, 3, 5, 7, 9]
8
9
[0, 2, 4]


### #1 괄호 검사

In [38]:
def checkBrackets(statement):
    stack = Stack()
    for ch in statement:
        if ch in ('{', '[', '('): # ('{[(') 이라고 해도 됨
            stack.push(ch)
        elif ch in ('}',']',')'):
            if stack.isEmpty():
                return False # 조건 2 위반
            else:
                left = stack.pop()
                if (ch=="}" and left != "{") or \  # 백슬래시(\) 문자로 줄바꿀 수 있다!
                (ch=="]" and left != "[") or \
                (ch==")" and left != "(") :
                    return False # 조건 3 위반
            
    return stack.isEmpty() # False이면 조건 1 위반

In [39]:
str = ("{A[(i+1)]=0;}", "A[(i+1])-0;")
for s in str:
    m = checkBrackets(s)
    print(s, "--->", m)

{A[(i+1)]=0;} ---> True
A[(i+1])-0; ---> False


### #2 수식의 계산(중위표기 및 후위표기 변환)

In [20]:
# 후위표기식의 결과 계산하기

def evalPostfix(expr):
    s = Stack()
    for token in expr :
        if token in "+-*/":
            val2 = s.pop()
            val1 = s.pop()
            if (token == '+'): s.push(val1+val2)
            elif (token == '-'): s.push(val1-val2)
            elif (token == '*'): s.push(val1*val2)
            elif (token == '/'): s.push(val1/val2)
        else:
            s.push(float(token))
    return s.pop()

In [21]:
expr1 = ['8','2','/','3','-','3','2','*','+']
print(expr1, ' ---> ', evalPostfix(expr1))

['8', '2', '/', '3', '-', '3', '2', '*', '+']  --->  7.0


In [29]:
# 중위표기식의 후위 변환 구현

def precedence(op): # 연산자의 우선순의 반환
    if op=='(' or op==')' : return 0
    elif op=='+' or op=='-' : return 1
    elif op=='*' or op=='/' : return 2 # 곱하기 나누기가 가장 높음
    else : return -1
    
def Infix2Postfix(expr): # 중위표기식 input
    s = Stack()
    output = [] # 후위표기식 출력 리스트
    
    for term in expr:
        if term in '(': # 왼쪽 괄호이면
            s.push('(') # 스택에 삽입
        elif term in ')': # 오른쪽 괄호이면
            while not s.isEmpty(): 
                op = s.pop()
                if op=='(' : break; # 왼쪽 괄호가 나올 때까지
                else : # 스택에서 연산자를 꺼내 출력
                    output.append(op)
        elif term in "+-*/": # 연산자이면
            while not s.isEmpty():
                op = s.peek()
                if (precedence(term) <= precedence(op)): # 우선순위가 높거나 같은 연산자를
                    output.append(op) # 스택에서 모두 꺼내 출력
                    s.pop()
                else:break
            s.push(term) # 마지막으로 현재 연산자 삽입
        else : # 피연산자이면
            output.append(term) # 바로 출력
    
    while not s.isEmpty(): # 처리가 끝났으면 스택에 남은 항목을
        output.append(s.pop()) # 모두 출력
    
    return output # 결과 반환

In [31]:
infix1 = ['8','/','2','-','3','+','(','3','*','2',')']
infix2 = ['1','/','2','*','4','*','(','1','/','4',')']
postfix1 = Infix2Postfix(infix1)
postfix2 = Infix2Postfix(infix2)
result1 = evalPostfix(postfix1)
result2 = evalPostfix(postfix2)

print(postfix1)
print(result1)
print(postfix2)
print(result2)

['8', '2', '/', '3', '-', '3', '2', '*', '+']
7.0
['1', '2', '/', '4', '*', '1', '4', '/', '*']
0.5


### #3 미로 탐색

In [50]:
map = [['1','1','1','1','1','1'],
       ['e','0','0','0','0','1'],
       ['1','0','1','0','1','1'],    
       ['1','1','1','0','0','x'],
       ['1','1','1','0','1','1'],
       ['1','1','1','1','1','1']]
MAZE_SIZE = 6

In [51]:
def isValidPos(x,y): # (x,y)가 갈 수 있는 방인지 검사하는 함수
    if x<0 or y<0 or x>=MAZE_SIZE or y >= MAZE_SIZE :
        return False # (x,y)가 미로 밖이면 -> 갈 수 없음
    else:
        return map[y][x] == '0' or map[y][x] == 'x' # 출구('x')이거나 방('0')이면 갈 수 있음
    
def DFS():
    stack = Stack()
    stack.push( (0,1) )
    print('DFS: ')
    
    while not stack.isEmpty(): # 공백이 아닐 동안
        here = stack.pop() # 항목을 꺼냄
        print(here, end='->')
        (x,y) = here # 스택에 저장된 튜플은 (x,y)순서임
        if (map[y][x] == 'x'): # 출구이면 탐색 성공. True 반환
            return True
        else :
            map[y][x] = '.' # 현재위치를 지나왔다고 '.' 표시
            # 4방향의 이웃을 검사해 갈 수 있으면 스택에 모두 삽입
            if isValidPos(x,y-1): stack.push( (x,y-1) ) # 상
            if isValidPos(x,y+1): stack.push( (x,y+1) ) # 하
            if isValidPos(x-1,y): stack.push( (x-1,y) ) # 좌
            if isValidPos(x+1,y): stack.push( (x+1,y) ) # 우
        print('현재 스택: ', stack.top) # 현재 스택 내용 출력
    return False # 탐색 실패. False 반환

result = DFS()
if result : print('미로탐색 성공')
else : print('미로탐색 실패')

DFS: 
(0, 1)->현재 스택:  [(1, 1)]
(1, 1)->현재 스택:  [(1, 2), (2, 1)]
(2, 1)->현재 스택:  [(1, 2), (3, 1)]
(3, 1)->현재 스택:  [(1, 2), (3, 2), (4, 1)]
(4, 1)->현재 스택:  [(1, 2), (3, 2)]
(3, 2)->현재 스택:  [(1, 2), (3, 3)]
(3, 3)->현재 스택:  [(1, 2), (3, 4), (4, 3)]
(4, 3)->현재 스택:  [(1, 2), (3, 4), (5, 3)]
(5, 3)->미로탐색 성공
